In [1]:
import sys
sys.path.append('/home/neel/Desktop/MOUS_hierarchical-representations/guslatho/syllabificator')
from syllabificator.main import syllabificate_word
import pandas as pd
import numpy as np


Starting Syllabificator...


2025-08-02 23:35:52.146486: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-02 23:35:52.827341: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2025-08-02 23:35:53.622580: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Libraries imported successfully.
To syllabify text, use syllabificate_text('Een bepaalde tekststring.')


In [3]:
print(sys.executable)

/home/neel/Desktop/MOUS_hierarchical-representations/.venv/bin/python


In [4]:
subtlex = pd.read_excel('/home/neel/Desktop/MOUS_hierarchical-representations/subtlex_v2_cleaned_no_drop3.xlsx') #failing entries have already been dropped actually, despite filename

From master_table IPA.ipynb: Number of entries failing each criterion (independently):

1) Invalid word pattern          : 153807
2) Exceeding length threshold    : 3055
3) Solely punctuation            : 37
4) SPEC with non-alphabetic chars: 106539
5) Single-letter + apostrophe    : 506
6) Entirely repeated characters  : 173
7) Triple consecutive letters    : 953

--- Final status counts ---
Status
keep    278061
drop    159442

In [5]:
subtlex

,Word,FREQcount,CDcount,FREQlow,CDlow,FREQlemma,SUBTLEXWF,Zipf,SUBTLEXCD,Lg10CD,dominant.pos,dominant.pos.freq,dominant.pos.lemma,dominant.pos.lemma.freq,all.pos,all.pos.freq,all.pos.lemma.freq,Status
0,ik,1744062,8054,778704,3125,1744527,39883.0334,7.597064,99.8017,3.9061,VNW,1743609,ik,1743944,.VNW.SPEC.N.VZ.,.1743609.448.4.1.,.1743944.448.134.1.,keep
1,je,1600888,8060,1315051,6535,1600923,36608.9449,7.559864,99.8761,3.9064,VNW,1600798,je,1600798,.VNW.SPEC.N.BW.LID.,.1600798.72.15.2.1.,.1600798.72.50.2.1.,keep
2,het,1068396,8066,780771,5578,1913811,24431.9717,7.384235,99.9504,3.9067,VNW,735390,het,735395,.VNW.LID.SPEC.WW.N.,.735390.332929.53.22.2.,.735395.332929.53.845403.31.,keep
3,de,1061177,8070,903872,6512,1063827,24266.8883,7.381291,100.0000,3.9069,LID,1060098,de,1062748,.LID.VNW.SPEC.VZ.,.1060098.806.272.1.,.1062748.806.272.1.,keep
4,dat,965424,8063,715570,6107,965431,22077.2184,7.340221,99.9133,3.9066,VNW,532576,dat,532576,.VNW.VG.SPEC.N.WW.,.532576.432794.51.2.1.,.532576.432794.51.9.1.,keep
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278056,aagghh,1,1,0,0,1,0.0229,1.656532,0.0124,0.3010,N,1,aagghh,1,.N.,.1.,.1.,keep
278057,aagezien,1,1,1,1,1,0.0229,1.656532,0.0124,0.3010,WW,1,aazien,1,.WW.,.1.,.1.,keep
278058,aaf,1,1,0,0,1,0.0229,1.656532,0.0124,0.3010,SPEC,1,aaf,1,.SPEC.,.1.,.1.,keep
278059,aada's,1,1,0,0,1,0.0229,1.656532,0.0124,0.3010,N,1,aada,1,.N.,.1.,.1.,keep


In [6]:
subtlex['Word'] = subtlex['Word'].str.lower()
subtlex['Syllabification'] = ''

In [ ]:
import re

def is_valid(word):
    return isinstance(word, str) and re.fullmatch(r"[a-zA-Zäëïöüáéíóúâêîôûç\-']+", word)

invalid_words = []
error_words = []

for i, row in subtlex.iterrows():
    word = row['Word']
    if is_valid(word):
        try:
            syll = syllabificate_word(word, alg='n', language='nl')
        except Exception as e:
            error_words.append((word, str(e)))
            syll = ''
    else:
        invalid_words.append(word)
        syll = ''
    subtlex.loc[i, 'Syllabification'] = syll

In [ ]:
subtlex.to_csv("SUBTLEX-cleaned_guslatho-syllabificator.csv")

In [ ]:
import pandas as pd

pd.Series(invalid_words).to_csv("invalid_words.csv", index=False)
pd.DataFrame(error_words, columns=["word", "error"]).to_csv("error_words.csv", index=False)